I have read the documentation for Gaussian mixture models, as this is the first time i have used it https://scikit-learn.org/stable/modules/mixture.html <br>
I have used ChatGPT to figure out why gmm was throwing errors and how to fix it (see comments below) <br>
I have looked up how to map gmm classes to actual classes, as stated in comments


In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.mixture import GaussianMixture as GMM
from scipy.stats import mode

In [2]:
header = ['M/F/C', 'SPKR', 'Phoneme-Number', 'Phoneme-Ascii', 'F0', 'F1', 'F2', 'F3']
df = pd.read_csv('./PetersonBarney/verified_pb.data', sep='\t', header=None, names=header)
#print(df)

X = df[['F0', 'F1', 'F2', 'F3']]
y = df['Phoneme-Number']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1)

In [3]:
# baseline model with center of gravity approach
centers = X_train.groupby(y_train).mean()

#print(centers)
from scipy.spatial.distance import cdist

distances = cdist(X_test, centers, metric='euclidean')

y_test_pred = np.argmin(distances, axis=1)


phoneme_classes = centers.index
y_test_pred_classes = phoneme_classes[y_test_pred]

test_accuracy = accuracy_score(y_test, y_test_pred_classes)
print(f'Test Accuracy for baseline model: {test_accuracy:.2f}')


Test Accuracy for baseline model: 0.49


In [4]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [5]:
# for supervised classifier, my choice is SVM
from sklearn.svm import SVC
svm = SVC(kernel='linear')  
svm.fit(X_train, y_train)

y_val_pred = svm.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f'SVM Validation Accuracy: {val_accuracy:.2f}')


svm.fit(X_train_scaled, y_train)
y_val_pred = svm.predict(X_val_scaled)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f'SVM Validation Accuracy using scaled data: {val_accuracy:.2f}')
# using scaled data does not seem to have a big impact

SVM Validation Accuracy: 0.90
SVM Validation Accuracy using scaled data: 0.89


In [6]:
# GMM as suggested in the slides, I think this could be optimized way better
# but the SVM is good enough for me
n_components = len(list(set(df['Phoneme-Number']))) # this is 10, confirmed by Data Wrangler extension in VSCode
gmm = GMM(n_components=n_components, covariance_type='full', random_state=1, init_params='k-means++')

gmm.fit(X_train)

y_train_pred = gmm.predict(X_train)

# Map each GMM component to the actual phoneme class by majority voting
# I have looked this up
component_to_class = {}
for component in np.unique(y_train_pred):
    mask = y_train_pred == component  # Get samples assigned to this component
    most_common = mode(y_train[mask])  # Get the mode (most common phoneme class)
    component_to_class[component] = most_common.mode

y_train_mapped = np.array([component_to_class[component] for component in y_train_pred])

train_accuracy = accuracy_score(y_train, y_train_mapped)
print(f"Train Accuracy: {train_accuracy:.2f}")

y_test_pred = gmm.predict(X_test)

y_test_mapped = np.array([component_to_class[component] for component in y_test_pred])

test_accuracy = accuracy_score(y_test, y_test_mapped) 
print(f"Test Accuracy: {test_accuracy:.2f}")


Train Accuracy: 0.42
Test Accuracy: 0.38
